In [28]:
import datetime as dt
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
month, day, year = 11, 5, 2019
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month='+str(month)+'&day='+str(day)+'&year='+str(year)
#url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=11&day=5&year=2019'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


In [62]:
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=1&day=20&year=2020'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
soup


<!DOCTYPE html>

<html class="no-js" data-root="/home/cbb/www" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201908281" rel="dns-prefetch"/>
<!-- no:cookie fast load the css.           -->
<script>function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === null?'':decodeURIComponent(re[1].replace(/\+/g,' '));}; document.srdev = gup('srdev')</script>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://d17lgqwvsissft.cloudfront.net" rel="preconnect"/>
<style>html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neue",helvetica,arial,sans-serif;color:#00

In [25]:
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=5&year=2019'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

<Response [200]>

In [3]:
soup.find('td').find('a').text

'Michigan State'

In [4]:
scores = soup.find_all('td')
#for score in scores:
#    print(score)


In [5]:
i=0
for score in scores:
    if i%6 in [0,3]:
        print(score.find('a').text)
    if i%6 in [1,4]:
        print(int(score.text))
    i+=1
    if i == 6:
        break

Michigan State
62
Kentucky
69


In [6]:
visitor_neutral = []
home_neutral = []
v_score = []
h_score =[]
dates = []
date =str(month)+'/'+str(day)+'/'+str(year)
i=0
for info in scores:
    if i%6 == 0:
        visitor_neutral.append(info.find('a').text)
    elif i%6 == 3:
        home_neutral.append(info.find('a').text)
    elif i%6 == 1:
        v_score.append(int(info.text))
    elif i%6 == 4:
        h_score.append(int(info.text))
    else:
        dates.append(date)
    i+=1

In [7]:
print(visitor_neutral[0], v_score[0], home_neutral[0], h_score[0])

Michigan State 62 Kentucky 69


In [8]:
df = pd.DataFrame(zip(dates, visitor_neutral, v_score, home_neutral, h_score), 
               columns =['Date','visitor_neutral', 'PTS','home_neutral','PTS']) 
df['Date']= pd.to_datetime(df['Date'])
df.head()

,Date,visitor_neutral,PTS,home_neutral,PTS
0,2019-11-05,Michigan State,62,Kentucky,69
1,2019-11-05,Kansas,66,Duke,68
2,2019-11-05,Louisville,87,Miami (FL),74
3,2019-11-05,North Florida,59,Florida,74
4,2019-11-05,Holy Cross,71,Maryland,95


In [68]:

month, day, year = 11, 5, 2019
current_date = dt.date(year,month,day)
visitor_neutral = []
home_neutral = []
v_score = []
h_score =[]
dates = []
master_df = pd.DataFrame()
date =str(month)+'/'+str(day)+'/'+str(year)
while(current_date != dt.date(2020, 3, 3)):
#reset lists
    visitor_neutral = []
    home_neutral = []
    v_score = []
    h_score =[]
    dates = []
    date =str(month)+'/'+str(day)+'/'+str(year)
#Get score page for date
    url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month='+str(month)+'&day='+str(day)+'&year='+str(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    scores = soup.find_all('td')
#get lists of all games / scores
    i=0
    for info in scores:
        if i%6 == 0:
            visitor_neutral.append(info.find('a').text)
        elif i%6 == 3:
            home_neutral.append(info.find('a').text)
        elif i%6 == 1:
            if not info.text:
                v_score.append(0)
            else:
                v_score.append(int(info.text))
        elif i%6 == 4:
            if not info.text:
                h_score.append(0)
            else:
                h_score.append(int(info.text))
        else:
            dates.append(date)
        i+=1
#make dataframe of lists
    df = pd.DataFrame(zip(dates, visitor_neutral, v_score, home_neutral, h_score), 
               columns =['Date','visitor_neutral', 'PTS','home_neutral','PTS']) 
    df['Date']= pd.to_datetime(df['Date'])
#combine with master dataframe
    master_df = pd.concat([master_df, df])
# increment day, month, year
    current_date += dt.timedelta(days=1)
    day = current_date.day
    month = current_date.month
    year = current_date.year

    

    

In [69]:
master_df.tail()

,Date,visitor_neutral,PTS,home_neutral,PTS
6,2020-03-02,Idaho State,78,Weber State,70
7,2020-03-02,Maryland-Eastern Shore,60,Howard,62
8,2020-03-02,Mississippi Valley State,61,Grambling,81
9,2020-03-02,Norfolk State,79,Delaware State,73
10,2020-03-02,North Carolina A&T,76,South Carolina State,65


In [20]:
master_df.head(1)

,Date,visitor_neutral,PTS,home_neutral,PTS
0,2019-11-05,Michigan State,62,Kentucky,69


In [22]:
master_df.tail(10)

,Date,visitor_neutral,PTS,home_neutral,PTS
5,2019-12-31,Evansville,52,Missouri State,65
6,2019-12-31,Florida A&M,70,Iowa State,68
7,2019-12-31,George Washington,51,Vermont,76
8,2019-12-31,Georgetown,60,Providence,76
9,2019-12-31,Hartford,68,Bowling Green State,81
10,2019-12-31,Miami (FL),73,Clemson,68
11,2019-12-31,Morgan State,82,Cal State Northridge,93
12,2019-12-31,Northern Iowa,70,Illinois State,76
13,2019-12-31,Rider,37,Wisconsin,65
14,2019-12-31,Temple,62,UCF,58
